# ReLax

> Recourse Explanation Library in JAX.

![Python](https://img.shields.io/pypi/pyversions/jax-relax.svg)
![CI status](https://github.com/BirkhoffG/jax-relax/actions/workflows/test.yaml/badge.svg)
![Docs](https://github.com/BirkhoffG/jax-relax/actions/workflows/deploy.yaml/badge.svg)
![pypi](https://img.shields.io/pypi/v/jax-relax.svg)
![GitHub License](https://img.shields.io/github/license/BirkhoffG/jax-relax.svg)

[**Overview**](#overview) | 
[**Installation**](#installation) | 
[**Tutorials**](https://birkhoffg.github.io/jax-relax/tutorials/getting_started.html) | 
[**Documentation**](https://birkhoffg.github.io/jax-relax/) | 
[**Citing ReLax**](#citing-relax) 


## Overview

`ReLax` (**Re**course Explanation **L**ibrary in J**ax**) is an efficient and scalable 
benchmarking library for recourse and counterfactual explanations, built on top of [jax](https://jax.readthedocs.io/en/latest/).
By leveraging language primitives such as *vectorization*, *parallelization*, and *just-in-time* compilation in 
[jax](https://jax.readthedocs.io/en/latest/),
`ReLax` offers massive speed improvements in generating individual (or local) explanations
for predictions made by Machine Learning algorithms.

Some of the key features are as follows:

* 🏃 **Fast and scalable** recourse generation.

* 🚀 **Accelerated** over `cpu`, `gpu`, `tpu`.

* 🪓  **Comprehensive** set of recourse methods implemented for benchmarking.

* 👐 **Customizable** API to enable the building of entire modeling and interpretation pipelines for new recourse algorithms.

## Installation

```bash
pip install jax-relax
# Or install the latest version of `jax-relax`
pip install git+https://github.com/BirkhoffG/jax-relax.git 
```

To futher unleash the power of accelerators (i.e., GPU/TPU), 
we suggest to first install this library via `pip install jax-relax`.
Then, follow steps in the [official install guidelines](https://github.com/google/jax#installation)
to install the right version for GPU or TPU.



## Dive into `ReLax`

`ReLax` is a recourse explanation library for explaining (any) JAX-based ML models.

In [ ]:
#| output: false
from relax.ml_model import MLModule
from relax.methods import VanillaCF
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import functools as ft
import jax

Let's first generate synthetic data:

In [ ]:
xs, ys = make_classification(n_samples=1000, n_features=10, random_state=42)
train_xs, test_xs, train_ys, test_ys = train_test_split(xs, ys, random_state=42)

Next, we fit an MLP model for this data. This model can be any model implmented in JAX. 
We will use the `MLModule` in `ReLax` as an example.

In [ ]:
#| output: false
model = MLModule()
model.train((train_xs, train_ys), epochs=5, batch_size=64)

Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 281ms/step - accuracy: 0.5562 - loss: 1.7768
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6167 - loss: 1.0861 
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6768 - loss: 0.8992 
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7343 - loss: 0.7246 
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7616 - loss: 0.5708 


<relax.ml_model.MLModule>

Generating recourse explanations are straightforward. 
We can simply call `generate_cf` of an implemented recourse method to generate *one* recourse explanation:

In [ ]:
#| output: false
vcf = VanillaCF()
vcf.init_fns()
cf = vcf.generate_cf(test_xs[0], model.pred_fn)
assert cf.shape == test_xs[0].shape

  0%|          | 0/100 [00:00<?, ?it/s]

Or generate a bunch of recourse explanations with `jax.vmap`:

In [ ]:
#| output: false
generate_fn = ft.partial(vcf.generate_cf, pred_fn=model.pred_fn)
cfs = jax.vmap(generate_fn)(test_xs)
assert cfs.shape == test_xs.shape

  0%|          | 0/100 [00:00<?, ?it/s]

## An End-to-End Example of using `ReLax`

See [Getting Started with ReLax](https://birkhoffg.github.io/ReLax/tutorials/getting_started.html).


## Supported Recourse Methods

| Method          | Type            | Paper Title                                                                                    | Ref                                     |
|-----------------|-----------------|------------------------------------------------------------------------------------------------|-----------------------------------------|
| `VanillaCF`     | Non-Parametric  | Counterfactual Explanations without Opening the Black Box: Automated Decisions and the GDPR.   | [[1]](https://arxiv.org/abs/1711.00399) |
| `DiverseCF`     | Non-Parametric  | Explaining Machine Learning Classifiers through Diverse Counterfactual Explanations.           | [[2]](https://arxiv.org/abs/1905.07697) |
| `ProtoCF`       | Semi-Parametric | Interpretable Counterfactual Explanations Guided by Prototypes.                                | [[3]](https://arxiv.org/abs/1907.02584) |
| `CounterNet`    | Parametric      | CounterNet: End-to-End Training of Prediction Aware Counterfactual Explanations.               | [[4]](https://arxiv.org/abs/2109.07557) |
| `GrowingSphere` | Non-Parametric  | Inverse Classification for Comparison-based Interpretability in Machine Learning.              | [[5]](https://arxiv.org/abs/1712.08443) |
| `CCHVAE`        | Semi-Parametric | Learning Model-Agnostic Counterfactual Explanations for Tabular Data.                          | [[6]](https://arxiv.org/abs/1910.09398) |
| `VAECF`         | Parametric      | Preserving Causal Constraints in Counterfactual Explanations for Machine Learning Classifiers. | [[7]](https://arxiv.org/abs/1912.03277) |
| `CLUE`          | Semi-Parametric | Getting a CLUE: A Method for Explaining Uncertainty Estimates.                                 | [[8]](https://arxiv.org/abs/2006.06848) |

: {tbl-colwidths="[17, 13, 65, 5]"}


## Citing `ReLax`

To cite this repository:

```latex
@software{relax2023github,
  author = {Hangzhi Guo and Xinchang Xiong and Amulya Yadav},
  title = {{R}e{L}ax: Recourse Explanation Library in Jax},
  url = {http://github.com/birkhoffg/ReLax},
  version = {0.1.0},
  year = {2023},
}
```